連接 drive

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/My Drive/Colab_Notebooks/HW3'
##進入專案目錄
os.chdir(path) #切換該目錄
os.listdir() #確認目錄內容

下載資料

In [ ]:
os.chdir(path + '/data')
!gdown '19b-RnXuAFhXhEHW7ah0M-pQsxHr1cPsZ'
!unzip -o hw3data.zip

執行 preprocess

In [ ]:
os.chdir(path + '/src')
!python preprocess.py -d ../data/train.jsonl -t ../cache/train.json && python preprocess.py -d ../data/public.jsonl -t ../cache/dev.json && python preprocess.py -d ../data/public.jsonl -t ../cache/test.json --no-ans

安裝模型訓練相依套件

In [ ]:
os.chdir(path + '/libs/summarization')
!pip install -r requirements.txt
!pip install transformers

訓練模型

In [ ]:
os.chdir(path + '/src')
!bash process.sh ../cache/train.json ../cache/dev.json ../model google/mt5-small

測試模型

In [ ]:
os.chdir(path + '/src')
!bash test.sh ../cache/test.json ../cache/dev.json ../cache google/mt5-small

In [ ]:
os.chdir(path + '/src')
!python postprocess.py -o ../cache/test.json -t ../cache/generated_predictions.txt -d ../cache/submission.jsonl

實驗生成策略

In [ ]:
import json
from transformers import pipeline
os.chdir(path + '/src')
summarizer = pipeline("summarization", model='../model', batch_size=4, device=0, max_length=20, do_sample=True, top_k=3)
with open('../cache/test.json') as f:
  for line in f:
    obj = json.loads(line)
    print(summarizer(obj['text']))

計分

In [ ]:
# 安裝套件
os.chdir(path + '/libs/rouge')
!pip install -e tw_rouge

In [ ]:
# 執行計分腳本
os.chdir(path + '/libs/rouge')
!python eval.py -r ../../data/public.jsonl -s ../../cache/submission.jsonl

繪圖

In [2]:
data = [{
  "rouge-1": {
    "r": 0.2239020253726327,
    "p": 0.2987549567901063,
    "f": 0.24952843916581152
  },
  "rouge-2": {
    "r": 0.08949925556786237,
    "p": 0.12071952767604702,
    "f": 0.10008353530115667
  },
  "rouge-l": {
    "r": 0.20256687618597122,
    "p": 0.2695475725925197,
    "f": 0.22540043410896224
  }
}
,
{
  "rouge-1": {
    "r": 0.22476110427652896,
    "p": 0.3004566690931736,
    "f": 0.2507010436437336
  },
  "rouge-2": {
    "r": 0.08871259372337557,
    "p": 0.11928301521387026,
    "f": 0.09911552887345927
  },
  "rouge-l": {
    "r": 0.20303821187372498,
    "p": 0.27045797464703414,
    "f": 0.22604723470224983
  }
}
,
{
  "rouge-1": {
    "r": 0.22275188793458928,
    "p": 0.30028125476729534,
    "f": 0.24892231678917565
  },
  "rouge-2": {
    "r": 0.08860175660888013,
    "p": 0.11976415732505273,
    "f": 0.09902741597367631
  },
  "rouge-l": {
    "r": 0.20166158930934372,
    "p": 0.2710960499423555,
    "f": 0.22496650302955473
  }
}
,
{
  "rouge-1": {
    "r": 0.21553722196587202,
    "p": 0.293080303658285,
    "f": 0.24177408488498417
  },
  "rouge-2": {
    "r": 0.08577774193560375,
    "p": 0.11773741049595449,
    "f": 0.09654427177741884
  },
  "rouge-l": {
    "r": 0.19509865169969245,
    "p": 0.26464649153544884,
    "f": 0.2184953716141435
  }
}
,
{
  "rouge-1": {
    "r": 0.2113629386602458,
    "p": 0.29343648112279913,
    "f": 0.23875507265500098
  },
  "rouge-2": {
    "r": 0.0832346246697365,
    "p": 0.11621860350486726,
    "f": 0.0941651851591165
  },
  "rouge-l": {
    "r": 0.19074807836602006,
    "p": 0.26460654143213796,
    "f": 0.215241818596132
  }
}
,
{
  "rouge-1": {
    "r": 0.20652853911589733,
    "p": 0.28863157407541584,
    "f": 0.23366701517472505
  },
  "rouge-2": {
    "r": 0.08193322662817831,
    "p": 0.11434995375189473,
    "f": 0.09257866640492828
  },
  "rouge-l": {
    "r": 0.18801950393935934,
    "p": 0.2623974749103417,
    "f": 0.21249726000408356
  }
}
,
{
  "rouge-1": {
    "r": 0.1963135930478937,
    "p": 0.2892483640134797,
    "f": 0.22606095683739738
  },
  "rouge-2": {
    "r": 0.07785764214082286,
    "p": 0.11312990480914012,
    "f": 0.08916023843703731
  },
  "rouge-l": {
    "r": 0.17882596907977544,
    "p": 0.2635090185724271,
    "f": 0.20579011925338506
  }
}
,
{
  "rouge-1": {
    "r": 0.010787018862867424,
    "p": 0.025906352620923594,
    "f": 0.014449524849434786
  },
  "rouge-2": {
    "r": 0.003296113840953794,
    "p": 0.007997363535223023,
    "f": 0.0044136199812842974
  },
  "rouge-l": {
    "r": 0.010312026676351555,
    "p": 0.02490165362295923,
    "f": 0.013834216615261793
  }
}]

In [17]:
def rouge_data(which_rouge,data):
  return {
      'rouge_r':list(reversed([epoch[which_rouge]['r'] for epoch in data])),
      'rouge_p':list(reversed([epoch[which_rouge]['p'] for epoch in data])),
      'rouge_f':list(reversed([epoch[which_rouge]['f'] for epoch in data])),
  }

In [ ]:
import matplotlib.pyplot as plt
x = [0,1,2,3,4,5,6,7]
plt.title("rouge-l curve for 7 epoch") # title
plt.ylabel("value") # y label
plt.xlabel("epoch") # x label
plt.plot(x,rouge_data('rouge-l',data)['rouge_r'],label='rouge_r')
plt.plot(x,rouge_data('rouge-l',data)['rouge_p'],label='rouge_p')
plt.plot(x,rouge_data('rouge-l',data)['rouge_f'],label='rouge_f')
plt.legend()
plt.show()